In [2]:
import scanpy as sc
import squidpy as sq
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import pandas as pd
from tqdm import tqdm
import os
import glob
import tifffile
sc.logging.print_header()

2023-04-06 22:24:43.141593: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 22:24:44.155730: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


scanpy==1.9.3 anndata==0.8.0 umap==0.5.3 numpy==1.22.4 scipy==1.9.1 pandas==1.5.3 scikit-learn==1.2.2 statsmodels==0.13.5 python-igraph==0.10.4 pynndescent==0.5.8


In [4]:
nanostring_dir = Path().resolve() / ".."/"datasets"/"cosmx"/"nanostring_data"
sample_dir = nanostring_dir / "Lung5_Rep2-Flat_files_and_images"


In [43]:
composite_folder = sample_dir / "CellComposite"
sample_img_file = glob.glob(str(composite_folder / "*"))[0]
img = Image.open(sample_img_file)
img_width = img.width
img_height = img.height

In [59]:
fov_pos_file = sample_dir / "Lung5_Rep2_fov_positions_file.csv"
fov_pos_df = pd.read_csv(str(fov_pos_file))

tbd = []
for i, (r) in enumerate(fov_pos_df.iloc):
    img_file = str(composite_folder / "CellComposite_F{:03.0f}.jpg".format(r.fov))
    if not os.path.exists(img_file):
        tbd.append(i)
    
fov_pos_df.drop(tbd, inplace=True)

global_min_x = fov_pos_df.x_global_px.min()
global_min_y = fov_pos_df.y_global_px.min()
global_max_x = fov_pos_df.x_global_px.max()
global_max_y = fov_pos_df.y_global_px.max()
global_width = int(0.5+global_max_x-global_min_x+img_width)
global_height = int(0.5+global_max_y-global_min_y+img_height)

In [60]:
buf = np.empty((global_height, global_width, 3), dtype=np.uint8)

In [62]:
for r in tqdm(fov_pos_df.iloc, total=len(fov_pos_df)):
    x = int(r.x_global_px-global_min_x)
    y = int(r.y_global_px-global_min_y)
    img_file = str(composite_folder / "CellComposite_F{:03.0f}.jpg".format(r.fov))
    if os.path.exists(img_file): 
        img = np.asarray(Image.open(img_file), dtype=np.uint8)
        img = img[::-1, :, :]
    
    buf[y:(y+img_height), x:(x+img_width), 0:3] = img[:(img_height), :(img_width), :]

100%|███████████████████████████████████████████| 30/30 [00:03<00:00,  8.58it/s]


In [63]:
tifffile.imwrite(str(sample_dir / "wsi_composite.tiff"), buf, photometric="rgb")